# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

In [1]:
#"topic" refers to a set of related words or terms that represent 
#a coherent theme or concept present in a collection of text documents
# Each topic is represented by a probability distribution over the vocabulary, 
#indicating the likelihood of each word occurring in that topic.
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Topic Modeling - Attempt #1 (All Text)

In [2]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('02D_dtm.pkl')
data

,aaababout,abandon,abandoned,abdomen,abercrombie,abigail,abilities,absentminded,absinthe,absolute,...,youtubers,yup,zac,zero,zipper,zoomed,zuckerberg,zurich,zuzus,éponine
Death and other,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Jacqueline,0,0,0,1,3,1,0,1,0,0,...,0,0,0,1,1,0,0,0,2,1
Kevin,0,3,1,0,0,0,0,0,1,0,...,1,0,0,0,0,1,2,0,0,0
Taylor,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The iron claw,0,0,0,0,0,0,0,0,0,3,...,0,1,1,0,0,0,0,0,0,0


In [3]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
#genism is used for topic modelling and finding similarity among large datasets
from gensim import matutils, models
#gives sparse matrix data structures as they are memory efficient
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()
#in tdm rows represent words and column represent documents

,Death and other,Jacqueline,Kevin,Taylor,The iron claw
aaababout,1,0,0,0,0
abandon,0,0,3,0,0
abandoned,0,0,1,1,0
abdomen,0,1,0,0,0
abercrombie,0,3,0,0,0


In [5]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
#converting it to genism compatible form
#sparse matrix
sparse_counts = scipy.sparse.csr_matrix(tdm)
#sparse matrix to genism convertible form
corpus = matutils.Sparse2Corpus(sparse_counts)
corpus

In [6]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("04D_cv.pkl", "rb"))
#converting in dictionary
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
id2word
#term id corresponding to word

{1246: 'detailsseason',
 1525: 'episode',
 4697: 'title',
 4756: 'tragicoriginal',
 3710: 'release',
 1689: 'february',
 2247: 'hulu',
 4755: 'tragic',
 4152: 'sixth',
 1245: 'details',
 362: 'begins',
 1949: 'glimpse',
 4109: 'showcasing',
 3872: 'rufuss',
 22: 'acclaim',
 1248: 'detective',
 3462: 'present',
 2300: 'imogenes',
 1506: 'entanglements',
 4477: 'sunil',
 1181: 'deepen',
 124: 'amidst',
 287: 'backdrop',
 1540: 'espionage',
 4495: 'surveillance',
 3153: 'orchestrated',
 1500: 'enigmatic',
 4953: 'viktor',
 3909: 'sams',
 2411: 'intricately',
 2620: 'lays',
 4025: 'series',
 1557: 'events',
 4878: 'unravel',
 3979: 'secrets',
 105: 'alliances',
 3784: 'revelation',
 4478: 'sunils',
 1726: 'financial',
 1237: 'desperation',
 2623: 'leading',
 4880: 'unsavory',
 104: 'alliance',
 4850: 'unexpected',
 2958: 'murder',
 97: 'alexandra',
 702: 'character',
 2385: 'intent',
 1614: 'exposing',
 1001: 'corporate',
 2763: 'malfeasance',
 3801: 'rich',
 4825: 'twists',
 5030: 'weaves

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [7]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.007*"kev" + 0.007*"hey" + 0.007*"kerry" + 0.006*"crowd" + 0.005*"cheering" + 0.004*"david" + 0.004*"von" + 0.004*"pam" + 0.004*"fritz" + 0.004*"son"'),
 (1,
  '0.011*"blow" + 0.010*"penis" + 0.005*"sort" + 0.005*"teeth" + 0.003*"viktor" + 0.003*"sams" + 0.003*"balls" + 0.003*"dick" + 0.003*"cock" + 0.003*"boyfriend"')]

In [8]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.009*"kev" + 0.005*"wee" + 0.005*"cos" + 0.004*"mate" + 0.004*"viktor" + 0.004*"sams" + 0.003*"quite" + 0.003*"son" + 0.003*"telly" + 0.003*"everybody"'),
 (1,
  '0.015*"blow" + 0.013*"penis" + 0.006*"teeth" + 0.006*"sort" + 0.005*"balls" + 0.004*"dick" + 0.004*"cock" + 0.003*"boyfriend" + 0.003*"bite" + 0.003*"rockhard"'),
 (2,
  '0.011*"hey" + 0.010*"kerry" + 0.010*"crowd" + 0.008*"cheering" + 0.007*"david" + 0.006*"von" + 0.006*"pam" + 0.006*"fritz" + 0.005*"erich" + 0.005*"friends"')]

In [9]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.012*"kev" + 0.007*"wee" + 0.006*"cos" + 0.006*"mate" + 0.004*"quite" + 0.004*"son" + 0.004*"telly" + 0.003*"everybody" + 0.003*"young" + 0.003*"bike"'),
 (1,
  '0.012*"hey" + 0.011*"kerry" + 0.010*"crowd" + 0.009*"cheering" + 0.007*"david" + 0.007*"von" + 0.006*"pam" + 0.006*"fritz" + 0.006*"erich" + 0.005*"friends"'),
 (2,
  '0.016*"blow" + 0.014*"penis" + 0.007*"sort" + 0.007*"teeth" + 0.005*"balls" + 0.005*"dick" + 0.004*"cock" + 0.004*"boyfriend" + 0.003*"bite" + 0.003*"rockhard"'),
 (3,
  '0.010*"viktor" + 0.010*"sams" + 0.005*"sail" + 0.005*"rufus" + 0.005*"imogene" + 0.004*"detective" + 0.004*"collier" + 0.004*"llewellyn" + 0.003*"hm" + 0.003*"worlds"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [10]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [11]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('02D_data_clean.pkl')
data_clean

,transcript
Death and other,death and other detailsseason episode episode...
Jacqueline,in get on your knees jacqueline novak transcen...
Kevin,kevin bridges the overdue catchup is a comedy...
Taylor,in her netflix standup comedy special have it...
The iron claw,the iron claw directed by sean durkinwritten b...


In [12]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
Death and other,death detailsseason episode episode title rele...
Jacqueline,get knees standup comedy show blend anecdotes ...
Kevin,kevin catchup comedy bridges stage backdrop co...
Taylor,comedy taylor tomlinson exploration life relat...
The iron claw,iron claw durkinwritten zac efron jeremy harri...


In [13]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people','youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
a_stop_words = list(text.ENGLISH_STOP_WORDS.union(add_stop_words))

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=a_stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.vocabulary_)
data_dtmn.index = data_nouns.index
data_dtmn

,death,detailsseason,episode,title,release,date,hulu,details,glimpse,showcasing,...,clarence,messiah,debt,teenager,activist,cases,destination,sparks,playboy,exes
Death and other,0,0,0,1,1,0,0,0,0,0,...,0,0,3,0,0,0,0,0,1,0
Jacqueline,1,3,1,0,1,0,1,1,0,0,...,1,0,1,0,0,1,1,0,0,1
Kevin,0,0,0,0,1,0,1,1,2,0,...,11,1,3,0,0,0,0,2,0,0
Taylor,0,0,0,0,0,0,2,0,0,1,...,1,0,4,0,0,0,0,0,0,0
The iron claw,0,0,0,0,1,3,0,0,0,0,...,0,0,3,1,1,0,0,0,0,0


In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"man" + 0.008*"kevin" + 0.008*"way" + 0.006*"world" + 0.006*"life" + 0.006*"hes" + 0.005*"gon" + 0.005*"door" + 0.005*"family" + 0.005*"bit"'),
 (1,
  '0.016*"blow" + 0.015*"job" + 0.014*"penis" + 0.014*"way" + 0.008*"cause" + 0.007*"sort" + 0.007*"kind" + 0.007*"life" + 0.006*"thing" + 0.006*"teeth"')]

In [16]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.019*"kevin" + 0.015*"way" + 0.011*"von" + 0.011*"kerry" + 0.010*"crowd" + 0.010*"world" + 0.009*"hey" + 0.009*"champion" + 0.009*"man" + 0.007*"family"'),
 (1,
  '0.015*"blow" + 0.014*"job" + 0.013*"penis" + 0.013*"way" + 0.008*"cause" + 0.007*"life" + 0.007*"kind" + 0.007*"sort" + 0.006*"thing" + 0.005*"teeth"'),
 (2,
  '0.009*"man" + 0.007*"bit" + 0.007*"life" + 0.006*"school" + 0.006*"friends" + 0.006*"hes" + 0.006*"guy" + 0.005*"wife" + 0.005*"gon" + 0.005*"night"')]

In [17]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.016*"kevin" + 0.013*"way" + 0.009*"world" + 0.009*"von" + 0.009*"kerry" + 0.009*"hey" + 0.009*"man" + 0.008*"crowd" + 0.008*"champion" + 0.007*"family"'),
 (1,
  '0.012*"friends" + 0.008*"theyre" + 0.008*"okay" + 0.007*"life" + 0.007*"lot" + 0.006*"friend" + 0.006*"parents" + 0.006*"man" + 0.006*"room" + 0.006*"talk"'),
 (2,
  '0.001*"way" + 0.001*"job" + 0.000*"penis" + 0.000*"blow" + 0.000*"life" + 0.000*"kevin" + 0.000*"cause" + 0.000*"man" + 0.000*"world" + 0.000*"night"'),
 (3,
  '0.012*"way" + 0.012*"blow" + 0.011*"job" + 0.011*"penis" + 0.008*"life" + 0.008*"bit" + 0.007*"man" + 0.006*"school" + 0.006*"cause" + 0.006*"kind"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [18]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [19]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
Death and other,death other detailsseason episode episode titl...
Jacqueline,get knees novak typical standup comedy show un...
Kevin,kevin overdue catchup comedy special kevin bri...
Taylor,netflix standup comedy special taylor tomlinso...
The iron claw,iron claw sean durkinwritten sean zac efron je...


In [20]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=a_stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.vocabulary_)
data_dtmna.index = data_nouns_adj.index
data_dtmna

,detailsseason,episode,title,tragicoriginal,release,date,february,hulu,sixth,details,...,teenager,unlikely,activist,landmark,cases,destination,sparks,jaded,playboy,exes
Death and other,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Jacqueline,1,3,1,0,1,0,0,1,1,0,...,0,1,0,0,0,1,1,0,0,2
Kevin,0,0,0,0,1,1,0,1,1,2,...,1,11,1,0,0,0,0,2,0,0
Taylor,0,0,0,0,0,0,0,2,0,0,...,0,1,0,0,0,0,0,0,0,0
The iron claw,0,0,0,0,1,0,3,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [21]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [22]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.011*"job" + 0.010*"blow" + 0.010*"penis" + 0.007*"cause" + 0.005*"theyre" + 0.005*"okay" + 0.005*"friends" + 0.004*"sort" + 0.004*"teeth" + 0.004*"single"'),
 (1,
  '0.010*"kevin" + 0.006*"crowd" + 0.006*"hey" + 0.005*"kerry" + 0.005*"door" + 0.005*"kev" + 0.005*"von" + 0.005*"bit" + 0.004*"son" + 0.004*"champion"')]

In [23]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"bit" + 0.006*"guy" + 0.006*"friends" + 0.005*"okay" + 0.005*"wee" + 0.005*"theyre" + 0.005*"kev" + 0.005*"wife" + 0.004*"door" + 0.004*"lot"'),
 (1,
  '0.019*"kevin" + 0.012*"crowd" + 0.011*"kerry" + 0.011*"hey" + 0.010*"von" + 0.008*"champion" + 0.008*"david" + 0.007*"erich" + 0.007*"pam" + 0.007*"fritz"'),
 (2,
  '0.014*"blow" + 0.013*"job" + 0.013*"penis" + 0.008*"cause" + 0.006*"sort" + 0.005*"teeth" + 0.004*"balls" + 0.004*"viktor" + 0.004*"dick" + 0.004*"sams"')]

In [24]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.013*"kevin" + 0.008*"crowd" + 0.007*"kerry" + 0.007*"door" + 0.007*"hey" + 0.007*"kev" + 0.006*"von" + 0.006*"bit" + 0.006*"son" + 0.005*"champion"'),
 (1,
  '0.011*"friends" + 0.010*"okay" + 0.008*"theyre" + 0.007*"single" + 0.006*"lot" + 0.006*"parents" + 0.005*"talk" + 0.005*"married" + 0.004*"job" + 0.004*"shes"'),
 (2,
  '0.015*"blow" + 0.014*"job" + 0.013*"penis" + 0.008*"cause" + 0.006*"sort" + 0.006*"teeth" + 0.004*"viktor" + 0.004*"balls" + 0.004*"dick" + 0.004*"sams"'),
 (3,
  '0.000*"kevin" + 0.000*"penis" + 0.000*"job" + 0.000*"crowd" + 0.000*"kerry" + 0.000*"blow" + 0.000*"door" + 0.000*"okay" + 0.000*"bit" + 0.000*"cause"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [25]:
# Our final LDA model (for now)
#increasing number of passes elads to better model converfgence
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.021*"kevin" + 0.013*"crowd" + 0.013*"kerry" + 0.012*"hey" + 0.011*"von" + 0.009*"champion" + 0.009*"david" + 0.008*"erich" + 0.008*"pam" + 0.007*"fritz"'),
 (1,
  '0.008*"bit" + 0.007*"wee" + 0.007*"wife" + 0.006*"kev" + 0.006*"door" + 0.005*"cos" + 0.005*"viktor" + 0.005*"guy" + 0.004*"sams" + 0.004*"old"'),
 (2,
  '0.000*"mistakes" + 0.000*"job" + 0.000*"huh" + 0.000*"dream" + 0.000*"whatd" + 0.000*"okay" + 0.000*"secrets" + 0.000*"kinda" + 0.000*"farm" + 0.000*"offer"'),
 (3,
  '0.013*"job" + 0.012*"blow" + 0.011*"penis" + 0.008*"cause" + 0.006*"okay" + 0.006*"theyre" + 0.006*"friends" + 0.005*"sort" + 0.005*"teeth" + 0.004*"single"')]

In [39]:
import pyLDAvis
import pyLDAvis.gensim
from gensim.corpora import Dictionary

# Create a Dictionary object from the corpus
id2wordna = Dictionary.from_corpus(corpusna, id2word=id2wordna)

# Enable the notebook mode for displaying the visualization
pyLDAvis.enable_notebook()

# Prepare the visualization
vis = pyLDAvis.gensim.prepare(ldana, corpusna, id2wordna, mds="mmds", R=30)

# Display the visualization
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.080241  0.138267       1        1  45.995986
1      0.084659 -0.116842       2        1  32.264132
0     -0.174188 -0.028088       3        1  21.728791
2      0.009288  0.006663       4        1   0.011091, topic_info=           Term       Freq      Total Category  logprob  loglift
1963      kevin  59.000000  59.000000  Default  30.0000  30.0000
2957      crowd  34.000000  34.000000  Default  29.0000  29.0000
829       penis  58.000000  58.000000  Default  28.0000  28.0000
3007      kerry  31.000000  31.000000  Default  27.0000  27.0000
3558        von  28.000000  28.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
413        lips   0.000313   2.916004   Topic4  -8.2945  -0.0326
488     wouldnt   0.000313   5.432165   Topic4  -8.2945  -0.6548
571        wont   0.000313   2.916004   Topic4  -8.2945  -0.0326
212     friends   0.000313  32.398469   Topic4  -8.2945  -2.4405
263   questions   0.000313  10.356353   Topic4  -8.2945  -1.3000

[237 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1181      1  1.003350       afraid
3561      3  0.772818     ambition
3618      3  0.957269    announcer
3568      3  0.772818  aspirations
323       1  0.653101          ass
...     ...       ...          ...
715       3  0.106046        young
3565      3  0.772818     youngest
1378      1  0.183629        youth
1378      2  0.826329        youth
3541      3  0.772818          zac

[330 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3])

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: profanity
* Topic 1: guns
* Topic 2: dogs,tweet
* Topic 3: mom, parents

In [27]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(1, 'Death and other'),
 (3, 'Jacqueline'),
 (1, 'Kevin'),
 (3, 'Taylor'),
 (0, 'The iron claw')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: profanity[Bill,Kevin,Mike]
* Topic 1: guns[Anthony,Jim,Joe,John]
* Topic 2: dogs,tweet[Gabriel,Dave,Ricky]
* Topic 3: mom, parents[Ali,Louis,Hasan]

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

### Assignment Solutions
1. Try further modifying the parameters of the topic models above and see if you can get better topics.

In [28]:
lda_model = models.LdaModel(corpus=corpusna, id2word=id2wordna, num_topics=5, passes=100, alpha=0.5, eta=0.01)
#alpha refers to prior belief of the topic distribution
#eta   refers to prior belief of word distribution

In [29]:
#Overall, these parameter adjustments may lead to more diverse and nuanced topics, 
#with documents and topics being influenced by a wider range of topics and words
lda_model.print_topics()

[(0,
  '0.029*"kevin" + 0.018*"crowd" + 0.017*"kerry" + 0.016*"hey" + 0.015*"von" + 0.013*"champion" + 0.012*"david" + 0.011*"erich" + 0.010*"pam" + 0.010*"fritz"'),
 (1,
  '0.016*"friends" + 0.015*"okay" + 0.012*"theyre" + 0.010*"single" + 0.008*"parents" + 0.008*"lot" + 0.007*"talk" + 0.007*"married" + 0.006*"date" + 0.006*"shes"'),
 (2,
  '0.013*"bit" + 0.011*"wee" + 0.010*"kev" + 0.010*"wife" + 0.009*"door" + 0.008*"cos" + 0.008*"guy" + 0.007*"old" + 0.006*"son" + 0.006*"mate"'),
 (3,
  '0.022*"viktor" + 0.020*"sams" + 0.009*"collier" + 0.008*"detective" + 0.007*"worlds" + 0.007*"factory" + 0.007*"hm" + 0.007*"imogene" + 0.007*"sail" + 0.006*"rufus"'),
 (4,
  '0.023*"blow" + 0.022*"penis" + 0.021*"job" + 0.012*"cause" + 0.010*"sort" + 0.009*"teeth" + 0.007*"balls" + 0.007*"dick" + 0.005*"cock" + 0.005*"problem"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: wife,mom
* Topic 1: profanity
* Topic 2: gun,kid,religion
* Topic 3: dogs,mexican
* Topic 4: parents,brown

In [30]:
# Extracting the topics associated with each document in the corpus
topic_assignments = []
for doc_topics in corpus_transformed:
    topic_assignments.append([topic for topic, _ in doc_topics])

# Pairing the topic assignments with document indices
list(zip(topic_assignments, data_dtmna.index))


[([1], 'Death and other'),
 ([3], 'Jacqueline'),
 ([1], 'Kevin'),
 ([3], 'Taylor'),
 ([0], 'The iron claw')]

* Topic 0: wife,mom [Anthony,Dave,Joe,John,Kevin]
* Topic 1: profanity [Jim,Ricky]
* Topic 2: gun,kid,religion [Bill]
* Topic 3: dogs,mexican [Gabriel,Mike]
* Topic 4: parents,brown [Ali,Hasan,Louis]

2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

* * ##### Topic Modeling - Attempt #4 (Nouns,Adjectives,AdVerbs and no proper nouns) 
* ##### [reference](https://sites.google.com/site/partofspeechhelp/home)

In [31]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj_verb(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj_verb = lambda pos: (pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'RB') and pos[:2] != 'NNP'
    tokenized = word_tokenize(text)
    nouns_adj_verb = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj_verb(pos)] 
    return ' '.join(nouns_adj_verb)

In [32]:
data_nouns_adj_verb = pd.DataFrame(data_clean.transcript.apply(nouns_adj_verb))
data_nouns_adj_verb

,transcript
Death and other,death other detailsseason episode episode titl...
Jacqueline,get knees novak typical standup comedy show un...
Kevin,kevin overdue catchup comedy special kevin bri...
Taylor,netflix standup comedy special taylor tomlinso...
The iron claw,iron claw sean durkinwritten sean zac efron je...


In [33]:
cvnav = CountVectorizer(stop_words=a_stop_words, max_df=.8)
data_cvnav = cvnav.fit_transform(data_nouns_adj_verb.transcript)
data_dtmnav = pd.DataFrame(data_cvnav.toarray(), columns=cvnav.vocabulary_)
data_dtmnav.index = data_nouns_adj_verb.index
data_dtmnav

,detailsseason,episode,title,tragicoriginal,release,date,february,hulu,sixth,details,...,teenager,unlikely,activist,landmark,cases,destination,sparks,jaded,playboy,exes
Death and other,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Jacqueline,1,3,1,0,1,0,0,1,1,1,...,0,1,0,0,0,1,1,0,0,2
Kevin,0,0,0,0,1,1,0,1,1,1,...,2,11,1,0,0,0,0,2,0,0
Taylor,0,0,0,0,0,0,0,1,2,0,...,0,1,0,0,0,0,0,0,0,0
The iron claw,0,0,0,0,1,0,3,4,0,0,...,0,0,0,1,1,0,0,0,0,0


In [34]:
corpusnav = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmnav.transpose()))

# Create the vocabulary dictionary
id2wordnav = dict((v, k) for k, v in cvnav.vocabulary_.items())

In [35]:
lda_nav = models.LdaModel(corpus=corpusnav, id2word=id2wordnav, num_topics=5, passes=100, alpha=0.5, eta=0.01)

In [36]:
lda_nav.print_topics()

[(0,
  '0.000*"blow" + 0.000*"kevin" + 0.000*"job" + 0.000*"bit" + 0.000*"penis" + 0.000*"guy" + 0.000*"crowd" + 0.000*"door" + 0.000*"cause" + 0.000*"okay"'),
 (1,
  '0.022*"haha" + 0.022*"crisis" + 0.022*"wok" + 0.000*"bit" + 0.000*"door" + 0.000*"wee" + 0.000*"kevin" + 0.000*"wife" + 0.000*"cos" + 0.000*"guy"'),
 (2,
  '0.022*"blow" + 0.020*"penis" + 0.020*"job" + 0.011*"cause" + 0.009*"sort" + 0.008*"teeth" + 0.006*"balls" + 0.006*"dick" + 0.005*"cock" + 0.005*"problem"'),
 (3,
  '0.013*"kevin" + 0.011*"okay" + 0.009*"hey" + 0.008*"crowd" + 0.008*"kerry" + 0.007*"von" + 0.006*"friends" + 0.006*"forever" + 0.006*"champion" + 0.005*"david"'),
 (4,
  '0.012*"bit" + 0.011*"wee" + 0.010*"kev" + 0.009*"wife" + 0.009*"door" + 0.008*"cos" + 0.007*"guy" + 0.007*"old" + 0.006*"quite" + 0.006*"telly"')]

In [37]:
corpus_transformed = lda_nav[corpusnav]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmnav.index))

[(3, 'Death and other'),
 (2, 'Jacqueline'),
 (4, 'Kevin'),
 (3, 'Taylor'),
 (3, 'The iron claw')]

* Topic 0: dogs,friend,mexican [Gabriel,Mike]
* Topic 1: profanity [Louis]
* Topic 2: gun,kid [Ali,Anthony,Bill,Jim,Ricky]
* Topic 3: mom,parents,gay [Dave,Hasan]
* Topic 4: wife,kid,mom [Joe,John,Kevin]

#### * [reference](https://towardsdatascience.com/latent-dirichlet-allocation-lda-a-guide-to-probabilistic-modeling-approach-for-topic-discovery-8cb97c08da3c)